In [58]:
import pandas as pd
from scipy import sparse
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

In [80]:
y_train = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\y_train.csv')

In [ ]:
y_train = y_train.dr

In [82]:
y_train.columns

Index(['Diabetes'], dtype='object')

In [5]:
y_test = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\y_test.csv')

In [10]:
x_train = sparse.load_npz(r'C:\Projects_ciência_dados\DIABETES\data\processed\x_train.npz')

In [11]:
x_test = sparse.load_npz(r'C:\Projects_ciência_dados\DIABETES\data\processed\x_test.npz')

#### Gaussian

In [14]:
from sklearn.naive_bayes import BernoulliNB

In [15]:
bernoulli = BernoulliNB()
bernoulli.fit(x_train, y_train)
previsao_bernoulli = bernoulli.predict(x_test)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
acuracia_bernoulli = accuracy_score(previsao_bernoulli, y_test)
print(f"Acurácia bernoulli: {acuracia_bernoulli}")

Acurácia bernoulli: 0.82


In [19]:
print(classification_report(previsao_bernoulli,y_test))

              precision    recall  f1-score   support

           0       0.81      0.86      0.84      1338
           1       0.83      0.77      0.80      1162

    accuracy                           0.82      2500
   macro avg       0.82      0.82      0.82      2500
weighted avg       0.82      0.82      0.82      2500



#### Árvores

In [31]:
parametros = {
    "max_depth":[100, 200, 500],
    "min_samples_split":[2,5,10],
    "min_samples_leaf":[1,5,10],
    "criterion":["gini", "entropy", "log_loss"]
}

In [25]:
from sklearn.tree import DecisionTreeClassifier

In [24]:
from sklearn.model_selection import GridSearchCV

#### Grid trabalha com dataset completo

In [32]:
grid = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=parametros)
grid.fit(x_train, y_train)
melhores_parametros = grid.best_params_
melhor_resultado = grid.best_score_

In [34]:
print(melhores_parametros)

{'criterion': 'gini', 'max_depth': 500, 'min_samples_leaf': 1, 'min_samples_split': 5}


In [35]:
print(melhor_resultado)

0.8242666666666667


In [21]:
arbol = DecisionTreeClassifier()
arbol.fit(x_train,y_train)
previsao_arbol = arbol.predict(x_test)

In [22]:
print(accuracy_score(previsao_arbol,y_test))

0.9884


In [23]:
print(classification_report(previsao_arbol,y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1411
           1       0.99      0.99      0.99      1089

    accuracy                           0.99      2500
   macro avg       0.99      0.99      0.99      2500
weighted avg       0.99      0.99      0.99      2500



#### Profundidade da árvore

In [37]:
arbol.get_depth()

556

#### Numero de folhas

In [38]:
arbol.get_n_leaves()

np.int64(582)

In [42]:
arbol.tree_

In [49]:
x_train_array = x_train.toarray()

In [50]:
x_train_DF = pd.DataFrame(x_train_array)

In [51]:
x_train_DF

,0,1,2,3,4,5,6,7,8,9,...,1870,1871,1872,1873,1874,1875,1876,1877,1878,1879
0,0.000000,2.000001,2.308478,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,4.940823,5.801497,6.415032,3.081867,2.519966,2.461312,3.917476,4.321749,2.000196,2.00109
1,0.000000,2.000001,2.308478,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,6.817569,5.163332,5.416465,4.822296,4.757269,3.508432,3.505110,5.160423,2.000196,2.00109
2,0.000000,2.000001,0.000000,2.308478,0.0,0.0,0.0,0.0,0.0,0.0,...,3.715192,6.091572,3.568303,4.767263,2.311969,0.525492,4.810935,2.785344,0.000000,0.00000
3,2.000001,0.000000,2.308478,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,6.740967,4.815243,3.812708,3.226329,3.245290,3.029307,4.261114,2.130872,0.000000,2.00109
4,0.000000,2.000001,2.308478,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,5.400434,3.596928,4.378329,2.841096,5.026599,0.653001,5.292028,4.930171,2.000196,2.00109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.000000,2.000001,2.308478,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,6.664366,4.815243,6.494172,4.801658,5.170597,1.050984,3.298927,3.519358,2.000196,0.00000
7496,2.000001,0.000000,0.000000,2.308478,0.0,0.0,0.0,0.0,0.0,0.0,...,4.251406,6.845767,4.231686,3.955521,2.754630,2.248797,3.436382,2.728130,0.000000,2.00109
7497,0.000000,2.000001,2.308478,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,3.638590,4.409138,6.889874,2.813579,2.645298,1.236452,3.367654,4.575724,0.000000,2.00109
7498,2.000001,0.000000,2.308478,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,3.791794,5.975542,5.344308,5.173134,2.087972,1.499198,3.161472,5.481379,2.000196,2.00109


##### Serve para ver as features mais importantes, mas ele perdeu o nome das features

In [52]:
importances = pd.DataFrame({
    "features":x_train_DF.columns,
    "importancia":arbol.feature_importances_
}).sort_values(by="importancia", ascending=False)

In [53]:
importances.head()

,features,importancia
1869,1869,0.230507
1873,1873,0.003681
1876,1876,0.002286
1092,1092,0.001967
815,815,0.001917


#### Regressão logística

In [54]:
from sklearn.linear_model import LogisticRegression

In [63]:
logistic = LogisticRegression()
logistic.fit(x_train, y_train)
previsao_logistic = logistic.predict(x_test)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [65]:
print(accuracy_score(previsao_logistic,y_test))

0.8948


In [66]:
print(classification_report(previsao_logistic,y_test))

              precision    recall  f1-score   support

           0       0.89      0.92      0.91      1377
           1       0.90      0.87      0.88      1123

    accuracy                           0.89      2500
   macro avg       0.89      0.89      0.89      2500
weighted avg       0.89      0.89      0.89      2500



In [83]:
cross_logistic = cross_val_score(logistic, x_train, y_train, cv=5)
cross_logistic.mean()

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning

np.float64(0.8695999999999999)

#### Random

In [84]:
from sklearn.ensemble import RandomForestClassifier

In [87]:
random = RandomForestClassifier()
random.fit(x_train,y_train)
previsao_random = random.predict(x_test)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [88]:
print(accuracy_score(previsao_random,y_test))

0.754
